Project: Process Market Orders<br>
Author: Vronsky Wikramanayake

- Task: Create process_orders (v2) that takes a list of market orders as bid's or offers & return an aggregated view, top 3 based on price, for BUY's & SELL's.<br>
- v2 Improvement: moved away from pandas for the main function, re wrote in raw python for efficinecy leveraging dict, and added pandas as optional allowing for better onward analytics.<br>
- Additional function: match_orders to match on price & return unmatched orders from each side, BUY & SELL. 
- Python: tested on 3.10

In [1]:
#several indipendant sublists of orders based on hourly buckets

orders0800 = [
    ('BUY', 11.60, 50),
    ('SELL', 11.60, 50),
    ('BUY', 11.60, 200),
    ('SELL', 12.00, 150),
    ('BUY', 11.60, 300),
    ('BUY', 10.50, 100),
    ('SELL', 21.75, 50),
    ('BUY', 10.50, 200),
    ('SELL', 12.00, 150),
    ('BUY', 10.60, 300),
    ('BUY', 9.50, 100),
    ('SELL', 12.00, 50),
    ('BUY', 5.50, 200),
    ('SELL', 11.60, 150),
    ('BUY', 9.60, 300)
]

orders0900 = [
    ('BUY', 8.60, 50),
    ('SELL', 11.60, 50),
    ('BUY', 11.30, 200),
    ('SELL', 12.00, 150),
    ('BUY', 11.60, 300),
    ('BUY', 10.90, 100),
    ('SELL', 21.75, 50),
    ('BUY', 10.50, 200),
    ('SELL', 12.00, 1500),
    ('BUY', 10.60, 300),
    ('BUY', 9.50, 100),
    ('SELL', 12.10, 50),
    ('BUY', 5.70, 200),
    ('SELL', 11.80, 150),
    ('BUY', 9.70, 3000)
]

ordersException = [
    ('HOLD', 8.60, 50)
]

In [2]:
buy_orders = {}
sell_orders = {}

def process_orders(orders):

    #isolate the elements inside a typical order & define them
    for order in orders:
        side, price, quantity = order
    
    #loop through the sublists of orders, distinguish between BUY vs SELL, append quantity if price has been added, if not create a new order dict
        if side == 'BUY':
            if price in buy_orders:
                buy_orders[price] += quantity
            else:
                buy_orders[price] = quantity
        elif side == 'SELL':
            if price in sell_orders:
                sell_orders[price] += quantity
            else:
                sell_orders[price] = quantity
        else: raise Exception("not a valid order")
    
    #order each side of the order book appropriately by price and only show top 3 so that any matches can be easily seen
    top3_buy_orders = dict(sorted(buy_orders.items(), key=lambda item: item[0], reverse=True)[:3])
    top3_sell_orders = dict(sorted(sell_orders.items(), key=lambda item: item[0], reverse=False)[:3])

    return top3_buy_orders, top3_sell_orders

In [11]:
#operate process_orders on a set of raw orders for 0800 & 0900, print the results

buy_orders, sell_orders = process_orders(orders0900)

print(buy_orders)
print(sell_orders)

{11.6: 1150, 11.3: 600, 10.9: 300}
{11.6: 50, 11.8: 450, 12.0: 5300}


In [4]:
# write a matching process that matches on price and reduces the quantity matched

matched_orders = []


def match_orders(buy_orders, sell_orders):
    
    buy_prices = sorted(buy_orders.keys(), reverse=True)
    sell_prices = sorted(sell_orders.keys())

    i, j = 0, 0
    while i < len(buy_prices) and j < len(sell_prices):
        buy_price, sell_price = buy_prices[i], sell_prices[j]

        if buy_price >= sell_price:
            # Match orders
            matched_quantity = min(buy_orders[buy_price], sell_orders[sell_price])
            matched_orders.append((sell_price, matched_quantity))

            # Reduce quantities in buy_orders and sell_orders
            buy_orders[buy_price] -= matched_quantity
            sell_orders[sell_price] -= matched_quantity

            # Remove entries with zero quantity
            if buy_orders[buy_price] == 0:
                del buy_orders[buy_price]
            if sell_orders[sell_price] == 0:
                del sell_orders[sell_price]

            # Move to the next price level
            if buy_orders.get(buy_price) == None:
                i += 1
            if sell_orders.get(sell_price) == None:
                j += 1
        else:
            break

    return matched_orders

In [12]:
matched_orders = match_orders(buy_orders, sell_orders)
print("Matched orders:", matched_orders)
print("Remaining buy orders:", buy_orders)
print("Remaining sell orders:", sell_orders)

Matched orders: [(11.6, 200), (11.6, 50), (11.6, 50), (11.6, 50)]
Remaining buy orders: {11.6: 1100, 11.3: 600, 10.9: 300}
Remaining sell orders: {11.8: 450, 12.0: 5300}


In [11]:
#if you did want to move to pandas for further analytics.

import pandas as pd

buy_df = pd.DataFrame({'Bid_Price': list(buy_orders.keys()), 'Bid_Quantity': list(buy_orders.values())})
sell_df = pd.DataFrame({'Offer_Price': list(sell_orders.keys()), 'Offer_Quantity': list(sell_orders.values())})

display(buy_df)
display(sell_df)

,Bid_Price,Bid_Quantity
0,11.6,850
1,11.3,200
2,10.9,100


,Offer_Price,Offer_Quantity
0,11.6,250
1,11.8,150
2,12.0,2000
